In [5]:
import oracledb

connection = oracledb.connect(
    user = 'BD150224426',
    password = 'Srzke6',
    dsn = '172.16.12.14/xe',
)


In [16]:
cursor = connection.cursor()


In [53]:
cursor = connection.cursor()

cursor.execute("""
                CREATE TABLE produto(
                nome  VARCHAR2(255) NOT NULL ,
                descricao VARCHAR2(255),
                codigo VARCHAR2(30) NOT NULL PRIMARY KEY,
                custo INTEGER NOT NULL,
                custoFixo INTEGER NOT NULL,
                comissao INTEGER NOT NULL,
                imposto INTEGER NOT NULL,
                rentabilidade INTEGER NOT NULL 
                )"""
)

In [56]:
cursor = connection.cursor()

cursor.execute(""" 
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caneta', 'Caneta Profissional', '1', 36, 15, 5, 12, 20)
                """)


cursor.execute(""" 
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caneta', 'Caneta Profissional', '3', 36, 15, 5, 12, 20)
                """)


connection.commit()

